In [1]:
import os
import time
import math
import random
import numpy as np
import pandas as pd
import torch
from torch import nn, einsum
import torch.nn.functional as F
import torch_optimizer as optim
from torch.utils.data import TensorDataset,DataLoader
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
from einops import rearrange
from einops.layers.torch import Rearrange

In [2]:
DEBUG = False

In [3]:
MODEL_NAME = 'transformer_v19'
base_dir = "./"
if not os.path.exists(f'models/{MODEL_NAME}'):
    os.makedirs(f'models/{MODEL_NAME}')

In [4]:
train_df = pd.read_csv(base_dir + 'train.csv')
test_df = pd.read_csv(base_dir + 'test.csv')

In [5]:
le = LabelEncoder()
train_df['pressure'] = le.fit_transform(train_df['pressure'])

In [6]:
train_df['RC'] = (train_df['R'].astype(str) + '_' + train_df['C'].astype(str))
train_df['RC'] = train_df['RC'].map({'20_50':0, '20_20':1, '50_20':2, '50_50':3, '5_50':4, '5_20':5, '50_10':6, '20_10':7, '5_10':8})
test_df['RC'] = (test_df['R'].astype(str) + '_' + test_df['C'].astype(str))
test_df['RC'] = test_df['RC'].map({'20_50':0, '20_20':1, '50_20':2, '50_50':3, '5_50':4, '5_20':5, '50_10':6, '20_10':7, '5_10':8})
# train_df['u_in_cat'] = train_df['u_in'].round().astype(int)
# test_df['u_in_cat'] = test_df['u_in'].round().astype(int)
train_df['u_in_0'] = (train_df['u_in'].round() == 0).astype(int)
test_df['u_in_0'] = (test_df['u_in'].round() == 0).astype(int)
train_df['u_in_5'] = (train_df['u_in'].round() == 5).astype(int)
test_df['u_in_5'] = (test_df['u_in'].round() == 5).astype(int)

In [7]:
# features = train_df.columns.drop(['id','breath_id','pressure'])

In [8]:
from sklearn.preprocessing import RobustScaler, StandardScaler
RS = StandardScaler()
all_df = pd.concat([train_df,test_df])
train_df['u_in'] = np.log1p(train_df['u_in'] - all_df['u_in'].min())
test_df['u_in'] = np.log1p(test_df['u_in'] - all_df['u_in'].min())
all_df['u_in'] = np.log1p(all_df['u_in'] - all_df['u_in'].min())

RS.fit(all_df[['u_in','time_step']])
train_df[['u_in','time_step']] = RS.transform(train_df[['u_in','time_step']])
test_df[['u_in','time_step']] = RS.transform(test_df[['u_in','time_step']])

# from sklearn.preprocessing import RobustScaler, StandardScaler
# RS = StandardScaler()
# all_df = pd.concat([train_df,test_df])
# RS.fit(all_df[['u_in','time_step']])
# train_df[['u_in','time_step']] = RS.transform(train_df[['u_in','time_step']])
# test_df[['u_in','time_step']] = RS.transform(test_df[['u_in','time_step']])

In [9]:
X_tr = train_df[['RC','u_in','u_out','u_in_0','u_in_5','time_step']].values.reshape(-1,80,6)
X_test = test_df[['RC','u_in','u_out','u_in_0','u_in_5','time_step']].values.reshape(-1,80,6)
# X_tr = train_df[features].values.reshape(-1,80,18)
# X_test = test_df[features].values.reshape(-1,80,18)

In [10]:
y_tr = train_df['pressure'].values.reshape(-1,80)

In [11]:
# pseudo_label = np.load("subs/sub_logits_transformer_v11.npy")
# from scipy.special import softmax
# pseudo_label = softmax(pseudo_label,axis=2)
# mask = test_df['u_out'].values.reshape(-1,80) == 0
# tmp1 = pseudo_label.max(2)
# res = []
# for i in tqdm(range(len(tmp1))):
#     res.append(tmp1[i][mask[i]].mean())
# res = np.array(res)
# mask = res > 0.6
# np.mean(mask)
# pseudo_label = pseudo_label.argmax(-1)
# X_tr = np.concatenate([X_tr,X_test[mask]],axis=0)
# y_tr = np.concatenate([y_tr,pseudo_label[mask]],axis=0)
# import gc
# del pseudo_label, mask, res, tmp1; gc.collect()

In [12]:
# unique_pressures = train_df["pressure"].unique()
# sorted_pressures = np.sort(unique_pressures)
# total_pressures_len = len(sorted_pressures)
# def find_nearest(prediction):
#     insert_idx = np.searchsorted(sorted_pressures, prediction)
#     if insert_idx == total_pressures_len:
#         # If the predicted value is bigger than the highest pressure in the train dataset,
#         # return the max value.
#         return sorted_pressures[-1]
#     elif insert_idx == 0:
#         # Same control but for the lower bound.
#         return sorted_pressures[0]
#     lower_val = sorted_pressures[insert_idx - 1]
#     upper_val = sorted_pressures[insert_idx]
#     return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val
# pseudo_label = pd.read_csv("pressure_submission.csv")['pressure'].values
# for i in range(len(pseudo_label)):
#     pseudo_label[i] = find_nearest(pseudo_label[i])
# pseudo_label = le.transform(pseudo_label).reshape(-1,80)
# X_tr = np.concatenate([X_tr,X_test],axis=0)
# y_tr = np.concatenate([y_tr,pseudo_label],axis=0)
# import gc
# del pseudo_label; gc.collect()

In [13]:
X_tr.shape,X_test.shape,y_tr.shape

((75450, 80, 6), (50300, 80, 6), (75450, 80))

In [14]:
if DEBUG:
    X_tr = X_tr[:1000]
    y_tr = y_tr[:1000]
    X_test = X_test[:1000]

In [15]:
def train_one_epoch(model, optimizer, train_dataloader, epoch, device = torch.device('cpu')):
    model.eval()
    MA_loss = 0
    count = 0
    for X,y in train_dataloader:
        X = X.to(device)
        y = y.to(device)
        y = F.one_hot(y,950).float()
        y[:,:,:-1] += 0.1 * y[:,:,1:]
        y[:,:,1:] += 0.1 * y[:,:,:-1]
        y[y==1] = 0.8
        
        optimizer.zero_grad()
        mask1 = X[:,:,2] == 0
        mask2 = X[:,:,2] == 1
        pred = model(X)

        pred = torch.sigmoid(pred[mask1].reshape(-1,950))
        y = y[mask1].reshape(-1,950)

        loss = -torch.sum(y * torch.log(1e-8 + pred) + (1-y) * torch.log(1 - pred + 1e-8),dim=-1).mean()
        loss.backward()
        optimizer.step()
        
        MA_loss += loss.item() * len(y)
        count += len(y)
    MA_loss /= count
    return MA_loss

def evaluation(model, val_dataloader, device = torch.device('cpu')):
    model.eval()
    criterion = nn.L1Loss()
    MA_loss = 0
    count = 0
    with torch.no_grad():
        for X,y in val_dataloader:
            X = X.to(device)
            y = y.to(device)
            mask = X[:,:,2] == 0
            pred = model(X)
            pred = torch.argmax(pred,dim=-1)
            pred = pred[mask].reshape(-1).cpu().numpy()
            pred = torch.Tensor(le.inverse_transform(pred)).to(device)
            y = y[mask].reshape(-1).cpu().long().numpy()
            y = torch.Tensor(le.inverse_transform(y)).to(device)
            loss = criterion(pred, y)
            # loss = criterion(pred.reshape(-1), y.reshape(-1))
            MA_loss += loss.item() * len(y)
            count += len(y)
        MA_loss /= count
    return MA_loss

def inference(model, test_dataloader, device = torch.device('cpu'), istest = False):
    model.eval()
    prediction = []
    with torch.no_grad():
        if istest:
            for X in test_dataloader:
                X = X[0]
                X = X.to(device)
                pred = model(X).cpu()
                pred = torch.argmax(pred,dim=-1)
                prediction.append(pred)
        else:
            for X, y in test_dataloader:
                X = X.to(device)
                pred = model(X).cpu()
                pred = torch.argmax(pred,dim=-1)
                prediction.append(pred)
    prediction = torch.cat(prediction,dim=0).numpy()
    prediction = le.inverse_transform(prediction.reshape(-1)).reshape(-1,80)
    return prediction

In [16]:
import torch
from torch import nn, einsum
import torch.nn.functional as F

from einops import rearrange
from einops.layers.torch import Rearrange

# helper functions

def exists(val):
    return val is not None

def default(val, d):
    return val if exists(val) else d

def calc_same_padding(kernel_size):
    pad = kernel_size // 2
    return (pad, pad - (kernel_size + 1) % 2)

# helper classes

class Swish(nn.Module):
    def forward(self, x):
        return x * x.sigmoid()

class GLU(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        out, gate = x.chunk(2, dim=self.dim)
        return out * gate.sigmoid()

class DepthWiseConv1d(nn.Module):
    def __init__(self, chan_in, chan_out, kernel_size, padding):
        super().__init__()
        self.padding = padding
        self.conv = nn.Conv1d(chan_in, chan_out, kernel_size, groups = chan_in)

    def forward(self, x):
        x = F.pad(x, self.padding)
        return self.conv(x)

# attention, feedforward, and conv module

class Scale(nn.Module):
    def __init__(self, scale, fn):
        super().__init__()
        self.fn = fn
        self.scale = scale

    def forward(self, x, **kwargs):
        return self.fn(x, **kwargs) * self.scale

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.fn = fn
        self.norm = nn.LayerNorm(dim)

    def forward(self, x, **kwargs):
        x = self.norm(x)
        return self.fn(x, **kwargs)

class FeedForward(nn.Module):
    def __init__(
        self,
        dim,
        mult = 4,
        dropout = 0.
    ):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim * mult),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim * mult, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class ConformerConvModule(nn.Module):
    def __init__(
        self,
        dim,
        causal = False,
        expansion_factor = 2,
        kernel_size = 31,
        dropout = 0.):
        super().__init__()

        inner_dim = dim * expansion_factor
        padding = calc_same_padding(kernel_size) if not causal else (kernel_size - 1, 0)

        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            Rearrange('b n c -> b c n'),
            nn.Conv1d(dim, inner_dim * 2, 1),
            GLU(dim=1),
            DepthWiseConv1d(inner_dim, inner_dim, kernel_size = kernel_size, padding = padding),
            nn.BatchNorm1d(inner_dim) if not causal else nn.Identity(),
            Swish(),
            nn.Conv1d(inner_dim, dim, 1),
            Rearrange('b c n -> b n c'),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [17]:
class CustomAttention(nn.Module):
    def __init__(
        self,
        dim,
        heads = 8,
        dim_head = 64,
        dropout = 0.,
        max_pos_emb = 512,
        causal = False
    ):
        super().__init__()
        inner_dim = dim_head * heads
        self.heads= heads
        self.scale = dim_head ** -0.5
        self.to_q = nn.Linear(dim, inner_dim, bias = False)
        self.to_kv = nn.Linear(dim, inner_dim * 2, bias = False)
        self.to_out = nn.Linear(inner_dim, dim)

        self.max_pos_emb = max_pos_emb
        # self.rel_pos_emb1 = nn.Linear(1, heads, bias=False)
        # self.rel_pos_emb2 = nn.Linear(1, heads, bias=False)
        self.rel_pos_emb = nn.Sequential(nn.Linear(1, dim_head),nn.GELU(),nn.Linear(dim_head, dim_head))
        self.dropout = nn.Dropout(dropout)

        self.causal = causal

    def forward(self, x, position, context = None, mask = None, context_mask = None):
        n, device, h, max_pos_emb, has_context = x.shape[-2], x.device, self.heads, self.max_pos_emb, exists(context)
        context = default(context, x)

        q, k, v = (self.to_q(x), *self.to_kv(context).chunk(2, dim = -1))
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), (q, k, v))

        dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale

        # shaw's relative positional embedding
        dist = rearrange(position, 'b i -> b i () ()') - rearrange(position, 'b j -> b () j ()')
        # rel_pos_emb = self.rel_pos_emb1(F.relu(dist)) + self.rel_pos_emb2(F.relu(-dist))   #(bijh)
        # pos_attn = rel_pos_emb.permute(0,3,1,2)
        rel_pos_emb = self.rel_pos_emb(dist)
        pos_attn = einsum('b h n d, b n r d -> b h n r', q, rel_pos_emb) * self.scale
        dots = dots + pos_attn

        if exists(mask) or exists(context_mask):
            mask = default(mask, lambda: torch.ones(*x.shape[:2], device = device))
            context_mask = default(context_mask, mask) if not has_context else default(context_mask, lambda: torch.ones(*context.shape[:2], device = device))
            mask_value = -torch.finfo(dots.dtype).max
            mask = rearrange(mask, 'b i -> b () i ()') * rearrange(context_mask, 'b j -> b () () j')
            dots.masked_fill_(~mask, mask_value)

        if self.causal:
            mask = torch.tril(torch.ones(dots.shape[-2:],device=dots.device)).T
            mask = rearrange(mask, 'n r -> () () n r')
            dots = dots - mask * 999

        attn = dots.softmax(dim = -1)

        out = einsum('b h i j, b h j d -> b h i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        out = self.to_out(out)
        return self.dropout(out)


class CustomConformerBlock(nn.Module):
    def __init__(
        self,
        *,
        dim,
        dim_head = 64,
        heads = 8,
        ff_mult = 4,
        conv_expansion_factor = 2,
        conv_kernel_size = 31,
        attn_dropout = 0.,
        ff_dropout = 0.,
        conv_dropout = 0.,
        causal = False
    ):
        super().__init__()
        self.ff1 = FeedForward(dim = dim, mult = ff_mult, dropout = ff_dropout)
        self.attn = CustomAttention(dim = dim, dim_head = dim_head, heads = heads, dropout = attn_dropout, causal = causal)
        self.conv = ConformerConvModule(dim = dim, causal = causal, expansion_factor = conv_expansion_factor, kernel_size = conv_kernel_size, dropout = conv_dropout)
        self.ff2 = FeedForward(dim = dim, mult = ff_mult, dropout = ff_dropout)

        self.attn = PreNorm(dim, self.attn)
        self.ff1 = Scale(0.5, PreNorm(dim, self.ff1))
        self.ff2 = Scale(0.5, PreNorm(dim, self.ff2))

    def forward(self, x, pos, mask = None):
        x = self.ff1(x) + x
        x = self.attn(x, position = pos, mask = mask) + x
        x = self.conv(x) + x
        x = self.ff2(x) + x
        return x

In [18]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len = 5000):
        super().__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self):
        return self.pe

class BrainModel(nn.Module):
    def __init__(self):
        super().__init__()
        DIM = 256
        n_layers = 4
        self.input_layer = nn.Sequential(nn.Linear(5,DIM),nn.Mish(),nn.Linear(DIM,DIM),nn.Mish())
        self.emb_RC = nn.Embedding(9,DIM)
        self.emb_u_in = nn.Embedding(101,DIM)
        self.scale_layer = nn.Linear(2*DIM,DIM)
        self.encoder = nn.ModuleList()
        for i in range(n_layers):
            self.encoder.append(CustomConformerBlock(dim = DIM,
                          dim_head = DIM//8,
                          heads = 8,
                          ff_mult = 4,
                          conv_expansion_factor = 2,
                          conv_kernel_size = 5,
                          attn_dropout = 0.1,
                          ff_dropout = 0.2,
                          conv_dropout = 0.05,
                          causal = False))
        self.fc = nn.Sequential(nn.Linear(DIM,DIM),nn.Mish(),nn.Linear(DIM,950))
        
    def forward(self, X):
        #(B,L,C)
        pos = X[:,:,-1]
        X_dense = self.input_layer(X[:,:,1:])
        X = torch.cat([X_dense, self.emb_RC(X[:,:,0].long())],dim=-1)
        X = self.scale_layer(X)
        for layer in self.encoder:
            X = layer(X,pos)
        y = self.fc(X)
        return y

In [19]:
N_EPOCHS = 100
N_FOLDS = 5
BATCH_SIZE = 128
oof_path = f"oofs/oof_finetune_{MODEL_NAME}"
device = torch.device('cuda:0')
kf = StratifiedKFold(N_FOLDS,shuffle=True, random_state=42)
oof = np.zeros(y_tr[:75450].shape)
y_true = le.inverse_transform(y_tr[:75450].reshape(-1)).reshape(y_tr[:75450].shape)
for fold, (train_index, val_index) in enumerate(kf.split(X_tr[:75450,0,0], X_tr[:75450,0,0])):
    print("fold:",fold)
    for RC in range(9):
      print("RC",RC)
      model_path = f'models/{MODEL_NAME}/model_{fold}_RC_{RC}.pt'

      train_index = np.concatenate([train_index,np.arange(75450,len(X_tr))])
      tr_idx = np.intersect1d(train_index,np.where(X_tr[:,0,0]==RC))
      val_idx = np.intersect1d(val_index,np.where(X_tr[:,0,0]==RC))
      test_idx = np.where(X_test[:,0,0]==RC)

      train_dataset = TensorDataset(torch.Tensor(X_tr[tr_idx]),torch.LongTensor(y_tr[tr_idx]))
      val_dataset = TensorDataset(torch.Tensor(X_tr[val_idx]),torch.LongTensor(y_tr[val_idx]))
      train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=2)
      val_dataloader = DataLoader(val_dataset, batch_size=3*BATCH_SIZE, shuffle=False, drop_last=False, num_workers=2)

      model = BrainModel().to(device)
      if (not os.path.exists(model_path)):
          model.load_state_dict(torch.load(f'models/{MODEL_NAME}/model_{fold}.pt'))
          optimizer = optim.Ranger(model.parameters(), lr=1e-4, weight_decay=0, alpha=0.5, k=5)
          scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, threshold=0.0001, min_lr=1e-6, verbose=True)
          best_score = evaluation(model, val_dataloader, device)
          torch.save(model.state_dict(),model_path)
          print(f"initial loss: {best_score}")
          stop_step = 0
          for epoch in tqdm(range(N_EPOCHS),leave=False):
              Train_loss = train_one_epoch(model, optimizer, train_dataloader, epoch, device)
              val_loss = evaluation(model, val_dataloader, device)
              scheduler.step(val_loss)
              if val_loss < best_score:
                best_score = val_loss
                torch.save(model.state_dict(),model_path)
                stop_step = 0
              else:
                stop_step += 1
              print(f"epoch {epoch}")
              print(f"Train_loss: {Train_loss}")
              print(f"val_loss: {val_loss}")
              print(f"best loss: {best_score}")
              print("*********************************")
              if stop_step > 20:
                break
      model.load_state_dict(torch.load(model_path))
      val_pred = inference(model, val_dataloader, device, False)
      oof[val_idx] = val_pred
    mask = X_tr[val_index,:,2]==0
    print(f"fold {fold}, score:",mean_absolute_error(y_true[val_index][mask], oof[val_index][mask]))
mask = X_tr[:75450,:,2]==0
print("CV score:",mean_absolute_error(y_true[mask], oof[mask]))
np.save(oof_path,oof)

fold: 0
RC 0
initial loss: 0.12861722443215698


C:\Users\yampa\.conda\envs\pytorch15\lib\site-packages\pytorch_ranger\ranger.py:172: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:1025.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


epoch 0
Train_loss: 2.6059817874583393
val_loss: 0.1278085380728838
best loss: 0.1278085380728838
*********************************
epoch 1
Train_loss: 2.5845617990994043
val_loss: 0.12747869371303228
best loss: 0.12747869371303228
*********************************
epoch 2
Train_loss: 2.5676203790743144
val_loss: 0.12765808772065432
best loss: 0.12747869371303228
*********************************
epoch 3
Train_loss: 2.555985259394525
val_loss: 0.12728918631803265
best loss: 0.12728918631803265
*********************************
epoch 4
Train_loss: 2.545664573881004
val_loss: 0.12702733848459516
best loss: 0.12702733848459516
*********************************
epoch 5
Train_loss: 2.5369464755436213
val_loss: 0.12711703404722877
best loss: 0.12702733848459516
*********************************
epoch 6
Train_loss: 2.5278598163107153
val_loss: 0.1268522940540507
best loss: 0.1268522940540507
*********************************
epoch 7
Train_loss: 2.5210798758976622
val_loss: 0.1269361959936364


epoch 61
Train_loss: 2.2967173423649303
val_loss: 0.12597995655008162
best loss: 0.12594234787558753
*********************************
Epoch    63: reducing learning rate of group 0 to 1.0000e-06.
epoch 62
Train_loss: 2.296484461973666
val_loss: 0.1261506569618938
best loss: 0.12594234787558753
*********************************
epoch 63
Train_loss: 2.2941959196284474
val_loss: 0.12601033367504208
best loss: 0.12594234787558753
*********************************
epoch 64
Train_loss: 2.295100433387619
val_loss: 0.12600019960646197
best loss: 0.12594234787558753
*********************************
epoch 65
Train_loss: 2.2945679431410366
val_loss: 0.12598862790453777
best loss: 0.12594234787558753
*********************************
epoch 66
Train_loss: 2.2947584086309325
val_loss: 0.12594812590632234
best loss: 0.12594234787558753
*********************************
epoch 67
Train_loss: 2.2942288762448073
val_loss: 0.12600454349456838
best loss: 0.12594234787558753
******************************

epoch 0
Train_loss: 2.4911415090743927
val_loss: 0.1310442881977918
best loss: 0.1310442881977918
*********************************
epoch 1
Train_loss: 2.4714728379431348
val_loss: 0.13118804751405777
best loss: 0.1310442881977918
*********************************
epoch 2
Train_loss: 2.45775638227505
val_loss: 0.13135261165388804
best loss: 0.1310442881977918
*********************************
epoch 3
Train_loss: 2.4472285906441047
val_loss: 0.13130343492078056
best loss: 0.1310442881977918
*********************************
epoch 4
Train_loss: 2.4382077170560503
val_loss: 0.1311993933831222
best loss: 0.1310442881977918
*********************************
epoch 5
Train_loss: 2.424423956232599
val_loss: 0.1311369759575361
best loss: 0.1310442881977918
*********************************
epoch 6
Train_loss: 2.421740737257437
val_loss: 0.13129396920009662
best loss: 0.1310442881977918
*********************************
epoch 7
Train_loss: 2.412827035915916
val_loss: 0.13129965454277434
best los

...

epoch 0
Train_loss: 2.1510847184341046
val_loss: 0.09130401241826007
best loss: 0.09130401241826007
*********************************
epoch 1
Train_loss: 2.1282990297050666
val_loss: 0.09127372957215377
best loss: 0.09127372957215377
*********************************
epoch 2
Train_loss: 2.11439966821368
val_loss: 0.0912957502025573
best loss: 0.09127372957215377
*********************************
epoch 3
Train_loss: 2.101455495106125
val_loss: 0.09131502097281999
best loss: 0.09127372957215377
*********************************
epoch 4
Train_loss: 2.0930662407854204
val_loss: 0.09129988018651249
best loss: 0.09127372957215377
*********************************
epoch 5
Train_loss: 2.081983921072521
val_loss: 0.09122968038565028
best loss: 0.09122968038565028
*********************************
epoch 6
Train_loss: 2.0767931262218897
val_loss: 0.09158480305591168
best loss: 0.09122968038565028
*********************************
epoch 7
Train_loss: 2.0668839503373326
val_loss: 0.0915228642896
be

epoch 0
Train_loss: 2.3483353422756776
val_loss: 0.13085893455654227
best loss: 0.13085893455654227
*********************************
epoch 1
Train_loss: 2.3166884273043427
val_loss: 0.13040904677736467
best loss: 0.13040904677736467
*********************************
epoch 2
Train_loss: 2.2957744383353713
val_loss: 0.13034513318293908
best loss: 0.13034513318293908
*********************************
epoch 3
Train_loss: 2.279309517393562
val_loss: 0.13034933725641232
best loss: 0.13034513318293908
*********************************
epoch 4
Train_loss: 2.2665682704403247
val_loss: 0.13053266057510549
best loss: 0.13034513318293908
*********************************
epoch 5
Train_loss: 2.2556201491102277
val_loss: 0.13060329469732126
best loss: 0.13034513318293908
*********************************
epoch 6
Train_loss: 2.2459974832695813
val_loss: 0.13047127432621955
best loss: 0.13034513318293908
*********************************
epoch 7
Train_loss: 2.2378153072970663
val_loss: 0.130402317972

epoch 62
Train_loss: 1.9666072268917296
val_loss: 0.12907703402773113
best loss: 0.12887437485108716
*********************************
epoch 63
Train_loss: 1.963075442418857
val_loss: 0.12902573711229198
best loss: 0.12887437485108716
*********************************
epoch 64
Train_loss: 1.9611523916209235
val_loss: 0.12936210335217144
best loss: 0.12887437485108716
*********************************
epoch 65
Train_loss: 1.9561748294930705
val_loss: 0.12900051260889053
best loss: 0.12887437485108716
*********************************
Epoch    67: reducing learning rate of group 0 to 1.0000e-05.
epoch 66
Train_loss: 1.9526620239308725
val_loss: 0.12910226410868558
best loss: 0.12887437485108716
*********************************
epoch 67
Train_loss: 1.9413193776792477
val_loss: 0.12922419629063675
best loss: 0.12887437485108716
*********************************
epoch 68
Train_loss: 1.9396076363506458
val_loss: 0.1289887346314115
best loss: 0.12887437485108716
*****************************

epoch 0
Train_loss: 2.264809517598096
val_loss: 0.13999152456526454
best loss: 0.13999152456526454
*********************************
epoch 1
Train_loss: 2.2496974734319215
val_loss: 0.14097845025924619
best loss: 0.13999152456526454
*********************************
epoch 2
Train_loss: 2.2405687763538844
val_loss: 0.14077227721760557
best loss: 0.13999152456526454
*********************************
epoch 3
Train_loss: 2.231676367624158
val_loss: 0.14014233465769643
best loss: 0.13999152456526454
*********************************
epoch 4
Train_loss: 2.2220032978421687
val_loss: 0.14052603293043972
best loss: 0.13999152456526454
*********************************
epoch 5
Train_loss: 2.213276079499444
val_loss: 0.14045158418004075
best loss: 0.13999152456526454
*********************************
epoch 6
Train_loss: 2.2063682255714356
val_loss: 0.14010797808315203
best loss: 0.13999152456526454
*********************************
epoch 7
Train_loss: 2.2014271825466913
val_loss: 0.14011751991880

epoch 0
Train_loss: 2.24332124418199
val_loss: 0.1320573631272611
best loss: 0.1320573631272611
*********************************
epoch 1
Train_loss: 2.2224780234381503
val_loss: 0.1325689936971083
best loss: 0.1320573631272611
*********************************
epoch 2
Train_loss: 2.211323987185134
val_loss: 0.1324069070082678
best loss: 0.1320573631272611
*********************************
epoch 3
Train_loss: 2.197756012815362
val_loss: 0.13224200278047968
best loss: 0.1320573631272611
*********************************
epoch 4
Train_loss: 2.1883291049109226
val_loss: 0.13203199229919838
best loss: 0.13203199229919838
*********************************
epoch 5
Train_loss: 2.179778395726114
val_loss: 0.13207004864630176
best loss: 0.13203199229919838
*********************************
epoch 6
Train_loss: 2.1752946905041406
val_loss: 0.13203622099385495
best loss: 0.13203199229919838
*********************************
epoch 7
Train_loss: 2.1627942228658563
val_loss: 0.13236744155933547
best 